In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
from meteostat import Stations, Daily
from datetime import datetime
import pandas as pd
import geopandas as gpd
from meteostat import Stations, Daily
from datetime import datetime

# Download weather station data

In [ ]:
import pandas as pd
import geopandas as gpd
from meteostat import Stations, Daily
from datetime import datetime
from shapely.geometry import Point
from geopy.distance import geodesic

# Load GADM region boundaries for Greece
greece_regions = gpd.read_file(
    "/Users/Thomas/Desktop/Skole/Business Analytics/Advanced Business Analytics/Data/gadm41_GRC_1.json"
)

# Fetch Greek weather stations
stations = Stations().region("GR").fetch()
stations_df = stations.reset_index()[['id', 'latitude', 'longitude']]

# Convert stations to GeoDataFrame
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df['longitude'], stations_df['latitude']),
    crs=greece_regions.crs
)

# Spatially join stations to regions
stations_with_regions = gpd.sjoin(
    stations_gdf,
    greece_regions[['NAME_1', 'geometry']],
    predicate='intersects',
    how='inner'
).rename(columns={'NAME_1': 'region'})

# Define time period
start = datetime(2000, 1, 1)
end = datetime(2024, 12, 31)

weather_list = []
weather_columns = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']

for station_id in stations_with_regions['id'].unique():
    try:
        weather = Daily(station_id, start, end).fetch()[weather_columns]
        if not weather.empty:
            weather['station_id'] = station_id
            weather['date'] = weather.index
            weather.reset_index(drop=True, inplace=True)
            weather_list.append(weather)
    except Exception as e:
        print(f"Error fetching station {station_id}: {e}")

# Combine data
weather_df = pd.concat(weather_list, ignore_index=True)

# Merge with region data
weather_with_regions = weather_df.merge(
    stations_with_regions[['id', 'region']],
    left_on='station_id',
    right_on='id',
    how='left'
)

# Aggregate data by region and date
region_weather = weather_with_regions.groupby(['region', 'date']).agg({
    'tavg': 'mean',
    'tmax': 'mean',
    'tmin': 'mean',
    'prcp': 'mean',
    'snow': 'mean',
    'wdir': 'mean',
    'wspd': 'mean',
    'wpgt': 'mean',
    'pres': 'mean',
    'tsun': 'mean'
}).reset_index()

# Rename to your preferred columns
region_weather.rename(columns={
    'tavg': 'temp_avg',
    'tmax': 'temp_max',
    'tmin': 'temp_min',
    'prcp': 'precip',
    'snow': 'snow_depth',
    'wdir': 'wind_dir',
    'wspd': 'wind_speed',
    'wpgt': 'wind_gust',
    'pres': 'pressure',
    'tsun': 'sunshine_duration'
}, inplace=True)

# ----------- FILL MISSING REGION-DATE USING NEAREST REGION ----------- #

# Generate full region-date index
dates_full = pd.date_range(start=start, end=end)
regions = region_weather['region'].unique()
full_index = pd.MultiIndex.from_product([regions, dates_full], names=['region', 'date'])

# Reindex to introduce missing rows
region_weather_full = region_weather.set_index(['region', 'date']).reindex(full_index).reset_index()

# Project to metric CRS for accurate centroids
greece_regions['region'] = greece_regions['NAME_1']
greece_projected = greece_regions.to_crs(epsg=3857)
greece_regions['centroid'] = greece_projected['geometry'].centroid

# Convert back to lat/lon for distance calcs
greece_regions['centroid_coords'] = greece_regions['centroid'].to_crs(epsg=4326).apply(lambda g: (g.y, g.x))
region_coords = greece_regions.set_index('region')['centroid_coords'].to_dict()

# Fill missing rows using closest region with data on same date
filled_rows = []
for idx, row in region_weather_full[region_weather_full.isnull().any(axis=1)].iterrows():
    target_region = row['region']
    date = row['date']
    target_coord = region_coords.get(target_region)
    if not target_coord:
        continue

    available = region_weather_full[
        (region_weather_full['date'] == date) &
        (region_weather_full['region'] != target_region)
    ].dropna()

    if available.empty:
        continue

    available = available.copy()
    available['dist'] = available['region'].apply(lambda r: geodesic(target_coord, region_coords[r]).km)
    closest = available.sort_values('dist').iloc[0]
    filled_row = row.copy()
    filled_row.update(closest.drop(['region', 'date', 'dist']))
    filled_rows.append(filled_row)

# Merge back filled rows
filled_df = pd.DataFrame(filled_rows)
region_weather_final = pd.concat([
    region_weather_full.dropna(),
    filled_df
], ignore_index=True).sort_values(['region', 'date']).reset_index(drop=True)

# Save final dataset
region_weather_final.to_csv(
    "/Users/Thomas/Desktop/Skole/Business Analytics/Advanced Business Analytics/Data/region_weather_2000_2024.csv",
    index=False
)

print(region_weather_final.head())




In [20]:
region_weather['region'].value_counts()

region
Aegean                          9132
Crete                           9132
Peloponnese,WesternGreeceand    9132
MacedoniaandThrace              9126
Attica                          9117
ThessalyandCentralGreece        8882
EpirusandWesternMacedonia       4129
Name: count, dtype: int64

In [41]:
df = pd.read_csv("/Users/Thomas/Desktop/Skole/Business Analytics/Advanced Business Analytics/Data/region_weather_epirus_patched.csv")

In [43]:
df['region'].value_counts()  

region
Aegean                          9132
Crete                           9132
Peloponnese,WesternGreeceand    9132
MacedoniaandThrace              9126
Attica                          9117
ThessalyandCentralGreece        8882
EpirusandWesternMacedonia       8398
Name: count, dtype: int64

In [35]:
import pandas as pd
from meteostat import Daily
from datetime import datetime

# List of fallback station IDs near Epirus
fallback_stations = [
    ('16642', 'Ioannina Airport'),
    ('16614', 'Kastoria Airport'),
    ('16632', 'Kozani Airport'),
    ('13629', 'Korca'),
    ('13625', 'Gjirokastra'),
    ('16613', 'Florina'),
    ('13580', 'Pretor-Pgc'),
    ('16641', 'Kerkyra Airport'),
    ('16648', 'Larissa Airport'),
    ('13583', 'Bitola')
]

# Define time range
start = datetime(2000, 1, 1)
end = datetime(2011, 9, 8)

# Placeholder for best patch
best_patch = None
best_station = None

# Try each fallback station
for station_id, station_name in fallback_stations:
    print(f"\n🔍 Trying station: {station_name} ({station_id})")
    try:
        patch_df = Daily(station_id, start, end).fetch()

        if not patch_df.empty:
            print(f"✅ Data found: {patch_df.shape[0]} rows")

            patch_df['date'] = patch_df.index
            patch_df['region'] = 'EpirusandWesternMacedonia'
            patch_df.reset_index(drop=True, inplace=True)

            patch_df.rename(columns={
                'tavg': 'temp_avg',
                'tmax': 'temp_max',
                'tmin': 'temp_min',
                'prcp': 'precip',
                'snow': 'snow_depth',
                'wdir': 'wind_dir',
                'wspd': 'wind_speed',
                'wpgt': 'wind_gust',
                'pres': 'pressure',
                'tsun': 'sunshine_duration'
            }, inplace=True)

            patch_columns = ['region', 'date', 'temp_avg', 'temp_max', 'temp_min',
                             'precip', 'snow_depth', 'wind_dir', 'wind_speed',
                             'wind_gust', 'pressure', 'sunshine_duration']

            patch_df = patch_df[patch_columns]

            best_patch = patch_df
            best_station = station_name
            break  # Stop at first with data

        else:
            print("❌ No data available.")

    except Exception as e:
        print(f"⚠️ Error fetching station {station_id}: {e}")

# Merge with region_weather if data was found
if best_patch is not None:
    print(f"\n🚀 Using patch from: {best_station}")

    region_weather_patched = pd.concat([region_weather, best_patch], ignore_index=True)
    region_weather_patched.drop_duplicates(subset=['region', 'date'], keep='last', inplace=True)
    region_weather_patched = region_weather_patched.sort_values(['region', 'date']).reset_index(drop=True)

    region_weather_patched.to_csv(
        "/Users/Thomas/Desktop/Skole/Business Analytics/Advanced Business Analytics/Data/region_weather_epirus_patched.csv",
        index=False
    )

    print("✅ Final patched dataset saved.")
else:
    print("❌ No suitable fallback station had data for the requested period.")


🔍 Trying station: Ioannina Airport (16642)
❌ No data available.

🔍 Trying station: Kastoria Airport (16614)
❌ No data available.

🔍 Trying station: Kozani Airport (16632)
❌ No data available.

🔍 Trying station: Korca (13629)
❌ No data available.

🔍 Trying station: Gjirokastra (13625)
❌ No data available.

🔍 Trying station: Florina (16613)
❌ No data available.

🔍 Trying station: Pretor-Pgc (13580)


❌ No data available.

🔍 Trying station: Kerkyra Airport (16641)
✅ Data found: 4269 rows

🚀 Using patch from: Kerkyra Airport
✅ Final patched dataset saved.
